In [12]:
!pip install --quiet gspread oauth2client plotly pandas

In [13]:
from google.colab import files
uploaded = files.upload()

Saving mood-tracker-460602-3314a779f9f9.json to mood-tracker-460602-3314a779f9f9 (1).json


In [31]:
# import libraries
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import plotly.express as px
import matplotlib.pyplot as plt

In [15]:
# get google sheets and google drive api running
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# load the json file
creds = ServiceAccountCredentials.from_json_keyfile_name("mood-tracker-460602-3314a779f9f9.json", scope)

# authorize the cliend
client = gspread.authorize(creds)

# open the moood tracker sheet
sheet = client.open("moodtracker").sheet1


In [28]:
# log time, mood, and note
def log_mood(mood, note):
    time = datetime.now().isoformat()
    sheet.append_row([time, mood, note])

In [26]:
# select mood
select_mood = widgets.Dropdown(
    options=["😊", "😠", "😕", "🎉"],
    description='Mood:',
    value="😊"
)

In [21]:
# option to add a note input
note = widgets.Text(
    placeholder='Add a short note...',
    description='Note:',
)

In [20]:
# submit button
submit_button = widgets.Button(description="Log Mood")

In [23]:
#  when submit -> logs these values
def on_submit(button):
    log_mood(select_mood.value, note.value)
    print(f"✅ Logged mood: {select_mood.value} | Note: {note.value}")

submit_button.on_click(on_submit)

In [29]:
# show widgets
display(select_mood, note, submit_button)

Dropdown(description='Mood:', index=1, options=('😊', '😠', '😕', '🎉'), value='😠')

Text(value='frustrated, overloaded with rx delays', description='Note:', placeholder='Add a short note...')

Button(description='Log Mood', style=ButtonStyle())

✅ Logged mood: 😠 | Note: frustrated, overloaded with rx delays
✅ Logged mood: 😊 | Note: on time with all rx prescriptions, team is working hard
✅ Logged mood: 🎉 | Note: amazing team progress
✅ Logged mood: 🎉 | Note: time for celebration!


In [37]:
# get df mood
def mood_chart():
    data = sheet.get_all_records()
    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"])
    df_today = df[df["time"].dt.date == datetime.today().date()]
    mood_counts = df_today["mood"].value_counts().reset_index()
    mood_counts.columns = ["mood", "count"]
    return mood_counts

In [38]:
# generate the dataframe
df_mood = mood_chart()

In [39]:
# plot
fig_plotly = px.bar(df_mood, x="mood", y="count", title="Today's Mood Trend (Plotly)")
fig_plotly.show()
